# Hosting Strands Agents with FireworksAI models in Amazon Bedrock AgentCore Runtime

This tutorial is adapted from [AWS AgentCore Sample tutorials](https://github.com/awslabs/amazon-bedrock-agentcore-samples/tree/main). For more details on AWS AgentCore please visit the [AgentCore docs](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/what-is-bedrock-agentcore.html)

## Overview

In this tutorial we will learn how to host your existing agent, using Amazon Bedrock AgentCore Runtime. 

We will focus on a Strands Agents example running Kimi K2 0905 on FireworksAI.

### Tutorial Details

| Information         | Details                                                                       |
|:--------------------|:------------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                                |
| Agent type          | Single                                                                        |
| Agentic Framework   | FireworksAI                                                                   |
| LLM model           | kimi-k2-instruct-0905                                                         |
| Tutorial components | Hosting agent on AgentCore Runtime. Using Strands Agent and FireworksAI Model |
| Tutorial vertical   | Cross-vertical                                                                |
| Example complexity  | Easy                                                                          |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3                                  |

### Tutorial Architecture

In this tutorial we will describe how to deploy an existing agent to AgentCore runtime.

In our example we will create a very simple code-generation agent which can:

1. Read files `file_read`
2. Create and write to files `file_write`
3. Write python code into those files `code_python`

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="60%"/>
</div>

### Tutorial Key Features

* Hosting Agents on Amazon Bedrock AgentCore Runtime
* Using FireworksAI models
* Using Strands Agents


## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker running

In [1]:
!make install

Installing dependencies...
uv pip install -r requirements.txt
Audited 11 packages in 116ms


## Creating your agents and experimenting locally

Before we deploy our agents to AgentCore Runtime, let's develop and run them locally for experimentation purposes.

For production agentic applications we will need to decouple the agent creation process from the agent invocation one. With AgentCore Runtime, we will decorate the invocation part of our agent with the `@app.entrypoint` decorator and have it as the entry point for our runtime. Let's first look how each agent is developed during the experimentation phase.

In [2]:
%%writefile strands_agents_fireworks_ai.py
from strands import Agent, tool
from strands_tools import file_read, file_write
import argparse
import json
# NOTE: FireworksAI is compatible with OpenAI sdk
from strands.models.openai import OpenAIModel
from dotenv import load_dotenv
import os

load_dotenv()

FIREWORKS_API_KEY = os.getenv("FIREWORKS_API_KEY")

@tool
def code_python(user_prompt: str):
    prompt = f"""
    You are a Python developer. You will be provided a user request for python code.
    Write clean, readable Python code.  Prioritize simple, working solutions.

    Only generate the code explicitly requested by the user. Do not include any additional code, if you need to add examples of how to run the code add it to the docstrings only.

    Remember the Zen of Python:
    ```
    Beautiful is better than ugly.
    Explicit is better than implicit.
    Simple is better than complex.
    Complex is better than complicated.
    Flat is better than nested.
    Sparse is better than dense.
    Readability counts.
    Special cases aren't special enough to break the rules.
    Although practicality beats purity.
    Errors should never pass silently.
    Unless explicitly silenced.
    In the face of ambiguity, refuse the temptation to guess.
    There should be one-- and preferably only one --obvious way to do it.
    Although that way may not be obvious at first unless you're Dutch.
    Now is better than never.
    Although never is often better than right now.
    If the implementation is hard to explain, it's a bad idea.
    If the implementation is easy to explain, it may be a good idea.
    Namespaces are one honking great idea – let's do more of those!
    ```

    The user request is: {user_prompt}
    """

    return prompt

model = OpenAIModel(
    client_args={
        "api_key": FIREWORKS_API_KEY,
        "base_url": "https://api.fireworks.ai/inference/v1",
    },
    # **model_config
    model_id="accounts/fireworks/models/kimi-k2-instruct-0905",
    params={
        "max_tokens": 3000,
        "temperature": 0.0,
    }
)

agent = Agent(
    model=model,
    tools=[file_read, file_write, code_python],
    system_prompt="You are a software engineer. You can read files, write files and generate python code."
)

def strands_agent_fireworks_ai(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = strands_agent_fireworks_ai(json.loads(args.payload))
    print(response)

Overwriting strands_agents_fireworks_ai.py


#### Invoking local agent

In [3]:
!python strands_agents_fireworks_ai.py '{"prompt": "Write a Python function to count letters in a string and return a dictionary with the count of each letter."}'

I'll write a Python function to count letters in a string and return a dictionary with the count of each letter.
Tool #1: code_python
```python
def count_letters(text):
    """
    Count the frequency of each letter in a string.
    
    Args:
        text (str): The input string to analyze
        
    Returns:
        dict: A dictionary with lowercase letters as keys and their counts as values
        
    Examples:
        >>> count_letters("Hello World!")
        {'h': 1, 'e': 1, 'l': 3, 'o': 2, 'w': 1, 'r': 1, 'd': 1}
        
        >>> count_letters("Python 3.9")
        {'p': 1, 'y': 1, 't': 1, 'h': 1, 'o': 1, 'n': 1}
        
        >>> count_letters("123 ABC abc")
        {'a': 2, 'b': 2, 'c': 2}
    """
    letter_counts = {}
    
    for char in text.lower():
        if char.isalpha():
            letter_counts[char] = letter_counts.get(char, 0) + 1
    
    return letter_counts
``````python
def count_letters(text):
    """
    Count the frequency of each letter in a stri

## Preparing your agent for deployment on AgentCore Runtime

Let's now deploy our agents to AgentCore Runtime. To do so we need to:
* Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
* Initialize the App in our code with `app = BedrockAgentCoreApp()`
* Decorate the invocation function with the `@app.entrypoint` decorator
* Let AgentCoreRuntime control the running of the agent with `app.run()`

In [4]:
%%writefile strands_agents_fireworks_ai.py
from strands import Agent, tool
from strands_tools import file_read, file_write
# NOTE: FireworksAI is compatible with OpenAI sdk
from strands.models.openai import OpenAIModel
from dotenv import load_dotenv
import os
from bedrock_agentcore.runtime import BedrockAgentCoreApp

app = BedrockAgentCoreApp()
load_dotenv()

FIREWORKS_API_KEY = os.getenv("FIREWORKS_API_KEY")

@tool
def code_python(user_prompt: str):
    prompt = f"""
    You are a Python developer. You will be provided a user request for python code.
    Write clean, readable Python code.  Prioritize simple, working solutions.

    Only generate the code explicitly requested by the user. Do not include any additional code, if you need to add examples of how to run the code add it to the docstrings only.

    Remember the Zen of Python:
    ```
    Beautiful is better than ugly.
    Explicit is better than implicit.
    Simple is better than complex.
    Complex is better than complicated.
    Flat is better than nested.
    Sparse is better than dense.
    Readability counts.
    Special cases aren't special enough to break the rules.
    Although practicality beats purity.
    Errors should never pass silently.
    Unless explicitly silenced.
    In the face of ambiguity, refuse the temptation to guess.
    There should be one-- and preferably only one --obvious way to do it.
    Although that way may not be obvious at first unless you're Dutch.
    Now is better than never.
    Although never is often better than right now.
    If the implementation is hard to explain, it's a bad idea.
    If the implementation is easy to explain, it may be a good idea.
    Namespaces are one honking great idea – let's do more of those!
    ```

    The user request is: {user_prompt}
    """

    return prompt

model = OpenAIModel(
    client_args={
        "api_key": FIREWORKS_API_KEY,
        "base_url": "https://api.fireworks.ai/inference/v1",
    },
    model_id="accounts/fireworks/models/kimi-k2-instruct-0905",
    params={
        "max_tokens": 3000,
        "temperature": 0.0,
    }
)

agent = Agent(
    model=model,
    tools=[file_read, file_write, code_python],
    system_prompt="You are a software engineer. You can read files, write files and generate python code."
)

@app.entrypoint
def strands_agent_fireworks_ai(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    response = agent(user_input)
    return response.message['content'][0]['text']


if __name__ == "__main__":
    app.run()

Overwriting strands_agents_fireworks_ai.py


## What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on the port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to the AWS standards

## Deploying the agent to AgentCore Runtime

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent. 

**Note:** Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this tutorial can will the Amazon Bedrock AgentCode Python SDK to easily package your artifacts and deploy them to AgentCore runtime.

### Configure AgentCore Runtime deployment

Next we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [5]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
from IPython.display import Markdown, display
import json
import boto3
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()

agent_name = "strands_fireworks_ai_getting_started"
response = agentcore_runtime.configure(
    entrypoint="strands_agents_fireworks_ai.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

Entrypoint parsed: file=/Users/robertobarroso/Desktop/repos/cookbook/integrations/AgentCore/strands_agents_fireworks_ai.py, bedrock_agentcore_name=strands_agents_fireworks_ai
Configuring BedrockAgentCore agent: strands_fireworks_ai_getting_started


⚠️  ℹ️  No container engine found (Docker/Finch/Podman not installed)
✅ Default deployment uses CodeBuild (no container engine needed)
💡 Run 'agentcore launch' for cloud-based building and deployment
💡 For local builds, install Docker, Finch, or Podman

Generated Dockerfile: /Users/robertobarroso/Desktop/repos/cookbook/integrations/AgentCore/Dockerfile
Generated .dockerignore: /Users/robertobarroso/Desktop/repos/cookbook/integrations/AgentCore/.dockerignore
Keeping 'strands_fireworks_ai_getting_started' as default agent
Bedrock AgentCore configured: /Users/robertobarroso/Desktop/repos/cookbook/integrations/AgentCore/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/Users/robertobarroso/Desktop/repos/cookbook/integrations/AgentCore/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/Users/robertobarroso/Desktop/repos/cookbook/integrations/AgentCore/Dockerfile'), dockerignore_path=PosixPath('/Users/robertobarroso/Desktop/repos/cookbook/integrations/AgentCore/.dockerignore'), runtime='None', region='us-west-2', account_id='023979239971', execution_role=None, ecr_repository=None, auto_create_ecr=True)

### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

In [8]:
from dotenv import load_dotenv
import os

load_dotenv()

FIREWORKS_API_KEY = os.getenv("FIREWORKS_API_KEY")
launch_result = agentcore_runtime.launch(
    env_vars={
        "OPENAI_API_BASE": "https://api.fireworks.ai/inference/v1",
        "OPENAI_API_KEY": FIREWORKS_API_KEY
    }
)

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'strands_fireworks_ai_getting_started' to account 023979239971 (us-west-2)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: strands_fireworks_ai_getting_started
✅ ECR repository available: 023979239971.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_fireworks_ai_getting_started
Getting or creating execution role for agent: strands_fireworks_ai_getting_started
Using AWS region: us-west-2, account ID: 023979239971
Role name: AmazonBedrockAgentCoreSDKRuntime-us-west-2-01b95285bc


✅ Reusing existing ECR repository: 023979239971.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_fireworks_ai_getting_started


✅ Reusing existing execution role: arn:aws:iam::023979239971:role/AmazonBedrockAgentCoreSDKRuntime-us-west-2-01b95285bc
✅ Execution role available: arn:aws:iam::023979239971:role/AmazonBedrockAgentCoreSDKRuntime-us-west-2-01b95285bc
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: strands_fireworks_ai_getting_started
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-01b95285bc
Reusing existing CodeBuild execution role: arn:aws:iam::023979239971:role/AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-01b95285bc
Using .dockerignore with 44 patterns
Uploaded source to S3: strands_fireworks_ai_getting_started/source.zip
Updated CodeBuild project: bedrock-agentcore-strands_fireworks_ai_getting_started-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.1s
🔄 PROVISIONING started (total: 1s)
✅ PROVISIONING completed in 8.8s
🔄 DOWNLOAD

### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [9]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: strands_fireworks_ai_getting_started


'READY'

### Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload. We will invoke the python developer agent with a simple leet code problem.

<div style="text-align:left">
    <img src="images/invoke.png" width=85%"/>
</div>

In [10]:
LEET_CODE_PROMPT = """
Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.
You may assume that each input would have exactly one solution, and you may not use the same element twice.
You can return the answer in any order.
"""

invoke_response = agentcore_runtime.invoke({"prompt": LEET_CODE_PROMPT})
invoke_response

{'ResponseMetadata': {'RequestId': '1a1d2600-3811-4665-98f3-76865bd0ef59',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 26 Sep 2025 00:50:41 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1a1d2600-3811-4665-98f3-76865bd0ef59',
   'baggage': 'Self=1-68d5e359-28adc6132ae389a42df36bee,session.id=6c89ee01-df08-4f71-86cc-90ecc931427c',
   'x-amzn-bedrock-agentcore-runtime-session-id': '6c89ee01-df08-4f71-86cc-90ecc931427c',
   'x-amzn-trace-id': 'Root=1-68d5e359-5c3dfcd513575b082864af5f;Self=1-68d5e359-28adc6132ae389a42df36bee'},
  'RetryAttempts': 0},
 'runtimeSessionId': '6c89ee01-df08-4f71-86cc-90ecc931427c',
 'traceId': 'Root=1-68d5e359-5c3dfcd513575b082864af5f;Self=1-68d5e359-28adc6132ae389a42df36bee',
 'baggage': 'Self=1-68d5e359-28adc6132ae389a42df36bee,session.id=6c89ee01-df08-4f71-86cc-90ecc931427c',
 'contentType': 'application/json',
 'statusCode': 200,
 'response': ['"```python\\n

In [13]:
from IPython.display import Markdown, display
import json

# Clean and display the response
full_text = json.loads(''.join(invoke_response['response']))
display(Markdown(full_text))

```python
def two_sum(nums, target):
    """
    Find two numbers in the array that add up to the target.
    
    Args:
        nums: List of integers
        target: Target sum
    
    Returns:
        List of two indices whose corresponding values add up to target
    
    Example:
        >>> two_sum([2, 7, 11, 15], 9)
        [0, 1]
        >>> two_sum([3, 2, 4], 6)
        [1, 2]
        >>> two_sum([3, 3], 6)
        [0, 1]
    """
    # Use a hash map to store numbers we've seen and their indices
    seen = {}
    
    for i, num in enumerate(nums):
        complement = target - num
        
        # Check if the complement exists in our hash map
        if complement in seen:
            return [seen[complement], i]
        
        # Store the current number and its index
        seen[num] = i
    
    # This should never be reached given the problem constraints
    return []


# Test cases
if __name__ == "__main__":
    # Test case 1
    nums1 = [2, 7, 11, 15]
    target1 = 9
    result1 = two_sum(nums1, target1)
    print(f"Input: nums = {nums1}, target = {target1}")
    print(f"Output: {result1}")
    print(f"Verification: {nums1[result1[0]]} + {nums1[result1[1]]} = {nums1[result1[0]] + nums1[result1[1]]}")
    print()
    
    # Test case 2
    nums2 = [3, 2, 4]
    target2 = 6
    result2 = two_sum(nums2, target2)
    print(f"Input: nums = {nums2}, target = {target2}")
    print(f"Output: {result2}")
    print(f"Verification: {nums2[result2[0]]} + {nums2[result2[1]]} = {nums2[result2[0]] + nums2[result2[1]]}")
    print()
    
    # Test case 3
    nums3 = [3, 3]
    target3 = 6
    result3 = two_sum(nums3, target3)
    print(f"Input: nums = {nums3}, target = {target3}")
    print(f"Output: {result3}")
    print(f"Verification: {nums3[result3[0]]} + {nums3[result3[1]]} = {nums3[result3[0]] + nums3[result3[1]]}")
    print()
    
    # Additional test case
    nums4 = [-1, 0, 1, 2]
    target4 = 1
    result4 = two_sum(nums4, target4)
    print(f"Input: nums = {nums4}, target = {target4}")
    print(f"Output: {result4}")
    print(f"Verification: {nums4[result4[0]]} + {nums4[result4[1]]} = {nums4[result4[0]] + nums4[result4[1]]}")
```

### Invoking AgentCore Runtime with boto3

Now that your AgentCore Runtime was created you can invoke it with any AWS SDK. For instance, you can use the boto3 `invoke_agent_runtime` method for it.

In [14]:
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "How much is 2X2?"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                print(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

2 × 2 = 4

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime created

In [15]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)